In [ ]:
import tensorflow as tf

from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential

from sklearn.model_selection import train_test_split

from helper_functions import plot_loss_accuracy_curves, plot_confusion_matrix

In [ ]:
# load the dataset
fashion_mninst = tf.keras.datasets.fashion_mnist
(X_train, y_train), (X_test, y_test) = fashion_mninst.load_data()

# Normalize the data
X_train, X_test = X_train / 255.0, X_test / 255.0

# Expand dimensions
X_train = tf.expand_dims(X_train, axis=-1)
X_test = tf.expand_dims(X_test, axis=-1)

# Convert tensors to NumPy arrays before splitting
X_train_np = X_train.numpy() if isinstance(X_train, tf.Tensor) else X_train
y_train_np = y_train.numpy() if isinstance(y_train, tf.Tensor) else y_train

# Split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_np, y_train_np, test_size=0.2, random_state=42)

# Convert back to tensors if needed
X_train = tf.convert_to_tensor(X_train)
X_val = tf.convert_to_tensor(X_val)
y_train = tf.convert_to_tensor(y_train)
y_val = tf.convert_to_tensor(y_val)

# Print shapes to verify
print('X_train shape:', X_train.shape)
print('X_val shape:', X_val.shape)
print('y_train shape:', y_train.shape)
print('y_val shape:', y_val.shape)


In [ ]:
# Build the model

model = Sequential([
    Input(shape=X_train[0].shape),
    Conv2D(32, kernel_size=(3, 3), activation='relu'), 
    MaxPool2D((2, 2)),
    Conv2D(64, kernel_size=(3, 3), activation='relu'), 
    MaxPool2D((2, 2)),
    Conv2D(128, kernel_size=(3, 3), activation='relu'),
    Dropout(0.3),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(10, activation='softmax')
])

model.summary()


In [ ]:
# Compile and fit the model

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

r = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50)

In [ ]:
# Plot the loss and Accuracy per iteration

plot_loss_accuracy_curves(r.history)

In [ ]:
# Evaluate the model
print('Test score:', model.evaluate(X_test, y_test))


In [ ]:
# Predict on test set
y_pred = model.predict(X_test).argmax(axis=1)

# Plot confusion matrix
plot_confusion_matrix(y_test, y_pred)